#### Code loads data from oviraptor catalogue. The goal is to generate a bootstrap sample of transit stellar hosts that has the same distribution of masses as the RV systems. A p-chip spline is fitted to the RV distribution in order to create a probability distribution (cdf). We draw weighted samples from the transit population with weights defined by the splinecdf. For each transiting planet, we draw probabilistic values using Chen and Kipping's mass forecaster. The masses, radii, and corresponding uncertainties are used as starting points for the process. After calculating system-level parameters (Q, mu, S, C, etc.), we perform KS and AD tests on each statistic and record their p-values. Finally, for each statistic, we determine the proportion of trials the null hypothesis is rejected and the proportion that the null hypothesis is accepted.

In [ ]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

import LMC
import archinfo

import astropy.stats
from astropy import constants as apc

import scipy
import scipy.signal as sig
import scipy.optimize as op
from scipy import stats
from scipy.optimize import curve_fit

import dcor
import random
import collections
import importlib as imp

import sys
sys.path.insert(0, "./forecaster/")
import mr_forecast as mr

import time
from timeit import default_timer

# define a few constants
pi = np.pi

MJUPITER = apc.M_jup.value
MEARTH = apc.M_earth.value
MSUN = apc.M_sun.value

RJUPITER = apc.R_jup.value
REARTH = apc.R_earth.value
RSUN = apc.R_sun.value

AU = apc.au.value
BIGG = apc.G.value

In [ ]:
# set manual controls and paths
NBOOT = 1000

# mr_relation can be 'chen-kipping', 'ning-ghosh', 'neil-rogers'
mr_relation = 'neil-rogers'

my_file = "C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_crossmatch_catalog_v2.csv"

In [ ]:
class System:
    def __init__(self, sysid=None):
        
        self.sysid = sysid

In [ ]:
raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[0]
VALUES = raw_data[2::2]

def get_key(keyname, keys = KEYS, values = VALUES):
    '''
    keyname must be a string
    '''
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return np.array(arr)

In [ ]:
# now make the data dictionary
d = {}
d['starname'] = get_key('hostname')
d['planet_id'] = get_key('pl_name')
d['rp'] = get_key('pl_rade')
d['rp_err1'] = get_key('pl_radeerr1')
d['rp_err2'] = get_key('pl_radeerr2')
d['mp'] = get_key('pl_bmasse')
d['mp_err1'] = get_key('pl_bmasseerr1')
d['mp_err2'] = get_key('pl_bmasseerr2')
d['rprs'] = get_key('pl_ratror')
d['rprs_err1'] = get_key('pl_ratrorerr1')
d['rprs_err2'] = get_key('pl_ratrorerr2')

d['ms'] = get_key('st_mass')
d['ms_err1'] = get_key('st_masserr1')
d['ms_err2'] = get_key('st_masserr2')
d['rs'] = get_key('st_rad')
d['rs_err1'] = get_key('st_raderr1')
d['rs_err2'] = get_key('st_raderr2')
d['p'] = get_key('pl_orbper')

d['K'] = get_key('pl_rvamp')
d['K_err1'] = get_key('pl_rvamperr1')
d['K_err2'] = get_key('pl_rvamperr2')

d['e'] = get_key('pl_orbeccen')
d['e_err1'] = get_key('pl_orbeccenerr1')
d['e_err2'] = get_key('pl_orbeccenerr2')

d['teff'] = get_key('st_teff')

# disc_method can vary within a system (i.e. discovery method for each planet)
d['disc_method'] = get_key('discoverymethod')

#'sys_disc_method' should be the same for all planets in a given system
d['sys_disc_method'] = get_key('system_disc_method')

In [ ]:
# eliminate systems with planets detected by different methods
keep = d['sys_disc_method'] != 'Mixed'
for k in d.keys():
    d[k] = d[k][keep]
print('Number of mixed planets: %d' %len(keep[keep==False]))

In [ ]:
# transit systems will not have values for these
d['K'][d['K']==''] = [np.nan]*len(d['K'][d['K']==''])
d['K_err1'][d['K_err1']==''] = [np.nan]*len(d['K_err1'][d['K_err1']==''])
d['K_err2'][d['K_err2']==''] = [np.nan]*len(d['K_err2'][d['K_err2']==''])

# radial velocity systems will not have values for these
d['rprs'][d['rprs']==''] = [np.nan]*len(d['rprs'][d['rprs']==''])
d['rprs_err1'][d['rprs_err1']==''] = [np.nan]*len(d['rprs_err1'][d['rprs_err1']==''])
d['rprs_err2'][d['rprs_err2']==''] = [np.nan]*len(d['rprs_err2'][d['rprs_err2']==''])

In [ ]:
# convert numerical quantities to floats; set missing values to np.nan
for k in d.keys():
    if (k[-11:] != "disc_method")*(k != "starname")*(k != 'planet_id'):
        
        arr = np.zeros(len(d["starname"]))

        for i, entry in enumerate(d[k]):
            if entry == "":
                arr[i] = np.nan
            else:
                arr[i] = float(entry)
                
        d[k] = np.copy(arr)

In [ ]:
# count up the planets in each system
d["npl"] = np.zeros(len(d["starname"]), dtype="int")

for i, s in enumerate(d["starname"]):
    d["npl"][i] = int(np.sum(d["starname"] == s))

# remove systems with fewer than 3 planets
keep = d["npl"] >= 3

for k in d.keys():
    d[k] = d[k][keep]
    
print("After removing systems with N < 3, {0} objects in {1} systems remain"
      .format(np.sum(keep), len(np.unique(d["starname"]))))

In [ ]:
# remove systems with missing stellar mass or radius
bad = np.isnan(d["ms"]) + np.isnan(d["rs"])

print("Removed the following systems with missing Mstar or Rstar")
for star in np.unique(d["starname"][bad]):
    print(star)   
    
for k in d.keys():
    d[k] = d[k][~bad]
    
print("\nAfter removals, {0} objects in {1} systems remain"
      .format(np.sum(~bad), len(np.unique(d["starname"]))))

In [ ]:
# fix one-sided Mstar uncertainties
for i, star in enumerate(d["starname"]):
    if np.isnan(d["ms_err1"][i]) + ~np.isnan(d["ms_err2"][i]):
        d["ms_err1"][i] = -d["ms_err2"][i]
    if ~np.isnan(d["ms_err1"][i]) + np.isnan(d["ms_err2"][i]):
        d["ms_err2"][i] = -d["ms_err1"][i]
        

# flag systems with no stellar mass uncertainty
bad = np.isnan(d["ms_err1"]) + np.isnan(d["ms_err2"])


print("The following systems have missing uncertainty on Mstar")
for star in np.unique(d["starname"][bad]):
    print(star)

In [ ]:
# fix one-sided Rstar uncertainties
for i, star in enumerate(d["starname"]):
    if np.isnan(d["rs_err1"][i]) + ~np.isnan(d["rs_err2"][i]):
        d["rs_err1"][i] = -d["rs_err2"][i]
    if ~np.isnan(d["rs_err1"][i]) + np.isnan(d["rs_err2"][i]):
        d["rs_err2"][i] = -d["rs_err1"][i]

        
# flag systems with no stellar radius uncertainty
bad = np.isnan(d["rs_err1"]) + np.isnan(d["rs_err2"])


print("The following systems have missing uncertainty on Rstar")
for star in np.unique(d["starname"][bad]):
    print(star)

In [ ]:
# remove systems with missing stellar mass or radius uncertainty
# or if their stellar mass or radius uncertainty
bad = np.isnan(d["rs_err1"]) + np.isnan(d["rs_err2"]) + \
        np.isnan(d["ms_err1"]) + np.isnan(d["ms_err2"])

for k in d.keys():
    d[k] = d[k][~bad]
    
print("\nAfter removing stars with missing mass/radius uncertainty, {0} objects in {1} systems remain"
      .format(np.sum(~bad), len(np.unique(d["starname"]))))

In [ ]:
# flag systems with only an upper/lower limit on planet mass or radius
bad  = np.isnan(d["rp_err1"]) * ~np.isnan(d["rp_err2"])
bad += ~np.isnan(d["rp_err1"]) * np.isnan(d["rp_err2"])

d["rp"][bad] = np.nan
d["rp_err1"][bad] = np.nan
d["rp_err2"][bad] = np.nan

bad  = np.isnan(d["mp_err1"]) * ~np.isnan(d["mp_err2"])
bad += ~np.isnan(d["mp_err1"]) * np.isnan(d["mp_err2"])

d["mp"][bad] = np.nan
d["mp_err1"][bad] = np.nan
d["mp_err2"][bad] = np.nan

print("Flagged, {0} planets with only upper/lower limit on mass or radius".format(np.sum(bad)))

In [ ]:
# flag planets with no mass uncertainty quoted
# only apply that flag for RV systems
bad = ~np.isnan(d["mp"]) * np.isnan(d["mp_err1"]) * np.isnan(d["mp_err2"]) * (d["sys_disc_method"]=="Radial Velocity")
d["mp"][bad] = np.nan


print("\nFlagged {0} planets with no MASS uncertainty quoted".format(np.sum(bad)))
print("The following systems host flagged planets:\n")

for i, star in enumerate(np.unique(d["starname"][bad])):
    print(star)

In [ ]:
# flag planets with no radius uncertainty quoted
# only apply that flag to transit systems
bad = ~np.isnan(d["rp"]) * np.isnan(d["rp_err1"]) * np.isnan(d["rp_err2"]) * (d['sys_disc_method'] == 'Transit')
d["rp"][bad] = np.nan

print("\nFlagged {0} planets with no RADIUS uncertainty quoted".format(np.sum(bad)))
print("The following systems host flagged planets:\n")

for i, star in enumerate(np.unique(d["starname"][bad])):
    print(star)

In [ ]:
# removes planets missing both a mass and radius (must have at least one)
# for now, flag planets and see what's going on 
bad = np.isnan(d["mp"])*np.isnan(d["rp"]) + \
(np.isnan(d['mp'])*(d['sys_disc_method']=='Radial Velocity')) + \
(np.isnan(d['rp'])*(d['sys_disc_method']=='Transit'))

print("Removed planets with neither mass nor radius:")
for planet in d['planet_id'][bad]:
    print(planet)
    
    
for k in d.keys():
    d[k] = d[k][~bad]
    
print("\nAfter removals, {0} objects in {1} systems remain"
      .format(np.sum(~bad), len(np.unique(d["starname"]))))

In [ ]:
# check for planets with large relative mass or radius uncertainties
mass_mu = np.copy(d["mp"])
mass_std = np.sqrt(d["mp_err1"]**2 + d["mp_err2"]**2)/np.sqrt(2)

radius_mu = np.copy(d["rp"])
radius_std = np.sqrt(d["rp_err1"]**2 + d["rp_err2"]**2)/np.sqrt(2)


# here's a histogram
fig, ax = plt.subplots(1,2, figsize=(18,5))
ax[0].hist(radius_std/radius_mu, bins=np.linspace(0,0.6,39), color="cornflowerblue")
ax[0].axvline(0.39, color="navy", lw=3)
ax[0].set_xlabel(r"$\sigma_r/r_p$", fontsize=20)
ax[1].hist(mass_std/mass_mu, bins=np.linspace(0,1.1,42), color="orange")
ax[1].axvline(1.0, color="red", lw=3)
ax[1].set_xlabel(r"$\sigma_m/m_p$", fontsize=20)
plt.show()


# remove the planets with large uncertainties
bad = (radius_std/radius_mu > 0.39) + (mass_std/mass_mu > 1.0)

print("Removed planets with large uncertainties from the following systems")
for star in np.unique(d["starname"][bad]):
    print(star)
    
for k in d.keys():
    d[k] = d[k][~bad]
    
print("\nAfter removals, {0} objects in {1} systems remain"
      .format(np.sum(~bad), len(np.unique(d["starname"]))))

# Enforce detection consistency between transit and RV samples

In [ ]:
# determine the nominal minimum radius of any planet discovered by RVs
# use Chen & Kipping (2017) to convert mass to radius

rv_mass = d["mp"][d["sys_disc_method"] == "Radial Velocity"]
rv_mass_err1 = d["mp_err1"][d["sys_disc_method"] == "Radial Velocity"]
rv_mass_err2 = d["mp_err2"][d["sys_disc_method"] == "Radial Velocity"]
rv_mass_err  = np.sqrt(rv_mass_err1**2 + rv_mass_err2**2)/np.sqrt(2)

bad = (rv_mass_err > 0.5*rv_mass) + np.isnan(rv_mass) + np.isnan(rv_mass_err)

rv_mass = rv_mass[~bad]
rv_mass_err = rv_mass_err[~bad]

rv_nominal_radius = np.zeros_like(rv_mass)
rv_stellar_radius = d["rs"][d["sys_disc_method"] == "Radial Velocity"][~bad]

for i, rvm in enumerate(rv_mass):
    rv_nominal_radius[i] = mr.Mstat2R(rvm, rv_mass_err[i], sample_size=1000)[0]

min_rv_radius = np.min(rv_nominal_radius[~np.isnan(rv_nominal_radius)])
    
nominal_rprs = (rv_nominal_radius*REARTH)/(rv_stellar_radius*RSUN)
min_rv_rprs = np.min(nominal_rprs[~np.isnan(nominal_rprs)])

plt.figure()
plt.plot(np.log10(rv_mass), np.log10(rv_nominal_radius), "k.")
plt.xlabel("log10(mass)", fontsize=20)
plt.ylabel("log10(radius)", fontsize=20)
plt.show()

In [ ]:
# remove very small transiting planets (that wouldn't be discovered by RV)
# remove very long period RV planets (that wouldn't be discovered by Kepler)
# don't throw out the NaN values, which indicate missing data (e.g no radius for some RV planets)

d['rprs'] = (d['rp']*REARTH)/(d['rs']*RSUN)

keep = (d["rprs"] >= min_rv_rprs) + ((np.isnan(d["rprs"]))+(d['rprs']==0) * (d["p"] < 1460.))
for k in d.keys():
    d[k] = d[k][keep]

    
# recount the planets in each system and eliminate systems reduced to fewer than 3 planets
for i, s in enumerate(d["starname"]):
    d["npl"][i] = int(np.sum(d["starname"] == s))
    
keep = d["npl"] >= 3

for k in d.keys():
    d[k] = d[k][keep]
    

print("After removing long period and small radius objects, {0} planets in {1} systems remain"
      .format(len(d["starname"]), len(np.unique(d["starname"]))))

In [ ]:
# recount the planets in each system
d["npl"] = np.zeros(len(d["starname"]), dtype="int")

for i, s in enumerate(d["starname"]):
    d["npl"][i] = int(np.sum(d["starname"] == s))
    

# remove systems with fewer than 3 planets
keep = d["npl"] >= 3

for k in d.keys():
    d[k] = d[k][keep]
    
print("After removing systems with N < 3, {0} objects in {1} systems remain"
      .format(np.sum(keep), len(np.unique(d["starname"]))))

# Statistics

In [ ]:
RV = d['sys_disc_method'] == 'Radial Velocity'
print('Here are the remaining %d RV systems:' \
      %len(np.unique(d['starname'][RV])))
print(np.unique(d['starname'][RV]))
print('%d radial velocity exoplanets' %len(d['planet_id'][RV]))

In [ ]:
transit = d['sys_disc_method'] == 'Transit'
kepler = np.flatnonzero(np.core.defchararray.find(d['planet_id'], 'Kepler')!=-1)
k2 = np.flatnonzero(np.core.defchararray.find(d['planet_id'], 'K2')!=-1)
tess = np.flatnonzero(np.core.defchararray.find(d['planet_id'], 'TOI')!=-1)

print('There are %d transit systems:' %len(np.unique(d['starname'][transit])))

print('%d Kepler planets in %d systems' %(len(d['starname'][kepler]), \
                            len(np.unique(d['starname'][kepler]))))
print('%d K2 planets in %d systems' %(len(d['starname'][k2]), \
                            len(np.unique(d['starname'][k2]))))
print('%d TESS planets in %d systems' %(len(d['starname'][tess]), \
                            len(np.unique(d['starname'][tess]))))

In [ ]:
# ensure that the rprs error for transit planets is > 0
for i in range(len(d['rprs'])):
    if d['sys_disc_method'][i] == 'Transit':
        if d['rprs'][i] == 0 or np.isnan(d['rprs'][i]):
            d['rprs'][i] = d['rp'][i]*REARTH/(d['rs'][i]*RSUN)
        if d['rprs_err1'][i] == 0 or np.isnan(d['rprs_err1'][i]):
            d['rprs_err1'][i] = d['rprs'][i]*np.sqrt((d['rp_err1'][i]/d['rp'][i])**2 + \
                                        (d['rs_err1'][i]/d['rs'][i])**2)
        if d['rprs_err2'][i] == 0 or np.isnan(d['rprs_err2'][i]):
            d['rprs_err2'][i] = d['rprs'][i]*np.sqrt((d['rp_err2'][i]/d['rp'][i])**2 + \
                                        (d['rs_err2'][i]/d['rs'][i])**2)
    if np.isnan(d['e'][i]):
        d['e'][i] = 0

# Read in masses/radii from Andrew

In [ ]:
NPL = len(d['K'])

if mr_relation == 'neil-rogers':
    mass_file = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/mr_predictions/masses_model3.csv'  # all transit
    radius_file = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/mr_predictions/radii_model3.csv' # all RV
    
    mass_prob_tr = np.zeros((NPL, NBOOT))
    radius_prob_rv = np.zeros_like(mass_prob_tr)
    transit_planets = []
    rv_planets = []
    
    raw_data = []
    with open(mass_file) as infile:
        reader = csv.reader(infile)
        for row in reader:
            raw_data.append(row)
    for i, value in enumerate(raw_data[1:]):
        transit_planets.append(value[0])
        mass_prob_tr[i] = np.array(value[2:], dtype=float)
        
    raw_data = []
    with open(radius_file) as infile:
        reader = csv.reader(infile)
        for row in reader:
            raw_data.append(row)
    for i, value in enumerate(raw_data[1:]):
        rv_planets.append(value[0])
        radius_prob_rv[i] = np.array(value[2:], dtype=float)
        
    # need to reject systems with all zeros
    mass_prob_tr = np.array(mass_prob_tr)
    radius_prob_rv = np.array(radius_prob_rv)
    transit_planets = np.array(transit_planets)
    rv_planets = np.array(rv_planets)
    
    bad = mass_prob_tr[:, 0]==0
    mass_prob_tr = mass_prob_tr[~bad]
    bad = radius_prob_rv[:, 0]==0
    radius_prob_rv = radius_prob_rv[~bad]
    
    # eliminate systems with RV/transit planets not in d
    bad = (d['sys_disc_method']=='Radial Velocity')*[name not in rv_planets for name in d['planet_id']]
    for key in d.keys():
        d[key] = d[key][~bad]
    bad = (d['sys_disc_method']=='Transit')*[name not in transit_planets for name in d['planet_id']]
    for key in d.keys():
        d[key] = d[key][~bad]

# Forecast masses/radii for all of the planets

In [ ]:
NPL = len(d['K'])

K_mu = np.copy(d['K'])
K_std = np.sqrt(d["K_err1"]**2 + d["K_err2"]**2)/np.sqrt(2)
      
rprs_mu = np.copy(d['rprs'])
rprs_std = np.sqrt(d["rprs_err1"]**2 + d["rprs_err2"]**2)/np.sqrt(2)

Mstar_mu = np.copy(d['ms'])
Mstar_std = np.sqrt(d["ms_err1"]**2 + d["ms_err2"]**2)/np.sqrt(2)

Rstar_mu = np.copy(d['rs'])
Rstar_std = np.sqrt(d["rs_err1"]**2 + d["rs_err2"]**2)/np.sqrt(2)

K_prob = np.zeros((NPL, NBOOT))
rprs_prob = np.zeros((NPL, NBOOT))
mpms_prob = np.zeros((NPL, NBOOT))
Mstar_prob = np.zeros((NPL, NBOOT))
Rstar_prob = np.zeros((NPL, NBOOT))
mass_prob = np.zeros((NPL, NBOOT))
radius_prob = np.zeros((NPL, NBOOT))

In [ ]:
for i in range(NPL):
    if i % 100 == 0:
        print(i)   
        
    if d['sys_disc_method'][i] == 'Radial Velocity' and ~np.isnan(d['K'][i]):
        K_prob[i] = stats.truncnorm(-K_mu[i]/K_std[i], (np.inf-K_mu[i])/K_std[i], 
                                       loc=K_mu[i], scale=K_std[i]).rvs(NBOOT)
        Mstar_prob[i] = stats.truncnorm((0.01-Mstar_mu[i])/Mstar_std[i], (10-Mstar_mu[i])/Mstar_std[i], 
                                       loc=Mstar_mu[i], scale=Mstar_std[i]).rvs(NBOOT)
        Rstar_prob[i] = stats.truncnorm((0.1-Rstar_mu[i])/Rstar_std[i], (5-Rstar_mu[i])/Rstar_std[i], 
                                       loc=Rstar_mu[i], scale=Rstar_std[i]).rvs(NBOOT)
        P = d['p'][i]*24*60*60
        for j in range(len(mpms_prob[i])):
            r = np.roots([2*pi*BIGG*Mstar_prob[i][j]*MSUN/(1-d['e'][i]**2)**1.5, -P*K_prob[i][j]**3, \
                                -2*P*K_prob[i][j]**3, -P*K_prob[i][j]**3])
            mpms_prob[i][j] = float(r[np.isreal(r)][0])
        mass_prob[i] = mpms_prob[i]*Mstar_prob[i]*MSUN/MEARTH
        if mr_relation == 'chen-kipping':
            radius_prob[i] = mr.Mpost2R(mass_prob[i])
        elif mr_relation == 'neil-rogers':
            radius_prob[i] = radius_prob_rv[rv_planets == d['planet_id'][i]] # insert radius probability from Neil
            
        
    elif d['sys_disc_method'][i] == 'Transit' and ~np.isnan(d['rprs'][i]):
        rprs_prob[i] = stats.truncnorm((0.01-rprs_mu[i])/rprs_std[i], (0.15-rprs_mu[i])/rprs_std[i], 
                                       loc=rprs_mu[i], scale=rprs_std[i]).rvs(NBOOT)
        Mstar_prob[i] = stats.truncnorm((0.01-Mstar_mu[i])/Mstar_std[i], (10-Mstar_mu[i])/Mstar_std[i], 
                                       loc=Mstar_mu[i], scale=Mstar_std[i]).rvs(NBOOT)
        Rstar_prob[i] = stats.truncnorm((0.1-Rstar_mu[i])/Rstar_std[i], (5-Rstar_mu[i])/Rstar_std[i], 
                                       loc=Rstar_mu[i], scale=Rstar_std[i]).rvs(NBOOT)
        radius_prob[i] = rprs_prob[i]*Rstar_prob[i]*RSUN/REARTH
        if mr_relation == 'chen-kipping':
            mass_prob[i] = mr.Rpost2M(radius_prob[i])
        elif mr_relation == 'neil-rogers': 
            mass_prob[i] = mass_prob_tr[transit_planets == d['planet_id'][i]] # insert mass probability from Neil
            
       
RV = d["sys_disc_method"] == "Radial Velocity"
TTV = ~np.isnan(d["mp"]) * ~RV

# redraw bad samples for chen-kipping relation
if mr_relation == 'chen-kipping':
    max_rv_mass = np.max(mass_prob[RV])
    min_rv_mass = np.min(mass_prob[RV])

    min_radius = np.min(radius_prob[radius_prob>0])
    min_mass = np.min(mass_prob[mass_prob>0])

    def check(mass, radius):
        # check for bad transit planets
        # or undetectable planets
        return mass > max_rv_mass or mass < min_rv_mass \
                or mass < min_mass or radius < min_radius 

    for i in range(len(rprs_prob)):
        if i % 100 == 0:
            print(i)    

        for j in range(len(rprs_prob[i])):
            bad = check(mass_prob[i][j], radius_prob[i][j])
            while bad == True and (~np.isnan(d['rprs'][i]) or ~np.isnan(d['K'][i])):
                if d['sys_disc_method'][i] == 'Transit':
                    rprs_prob[i][j] = stats.truncnorm((0.01-rprs_mu[i])/rprs_std[i], (0.15-rprs_mu[i])/rprs_std[i], 
                                               loc=rprs_mu[i], scale=rprs_std[i]).rvs(1)[0]
                    Mstar_prob[i][j] = stats.truncnorm((0.01-Mstar_mu[i])/Mstar_std[i], (10-Mstar_mu[i])/Mstar_std[i], 
                                               loc=Mstar_mu[i], scale=Mstar_std[i]).rvs(1)[0]
                    Rstar_prob[i][j] = stats.truncnorm((0.1-Rstar_mu[i])/Rstar_std[i], (5-Rstar_mu[i])/Rstar_std[i], 
                                               loc=Rstar_mu[i], scale=Rstar_std[i]).rvs(1)[0]
                    radius_prob[i][j] = rprs_prob[i][j]*Rstar_prob[i][j]*RSUN/REARTH
                    mass_prob[i][j] = mr.Rpost2M([radius_prob[i][j]])

                if d['sys_disc_method'][i] == 'Radial Velocity':
                    K_prob[i][j] = stats.truncnorm(-K_mu[i]/K_std[i], (np.inf-K_mu[i])/K_std[i], 
                                           loc=K_mu[i], scale=K_std[i]).rvs(1)[0]
                    Mstar_prob[i][j] = stats.truncnorm((0.01-Mstar_mu[i])/Mstar_std[i], (10-Mstar_mu[i])/Mstar_std[i], 
                                                   loc=Mstar_mu[i], scale=Mstar_std[i]).rvs(1)[0]
                    Rstar_prob[i][j] = stats.truncnorm((0.1-Rstar_mu[i])/Rstar_std[i], (5-Rstar_mu[i])/Rstar_std[i], 
                                                   loc=Rstar_mu[i], scale=Rstar_std[i]).rvs(1)[0]
                    P = d['p'][i]*24*60*60
                    r = np.roots([2*pi*BIGG*Mstar_prob[i][j]*MSUN/(1-d['e'][i]**2)**1.5, -P*K_prob[i][j]**3, \
                                            -2*P*K_prob[i][j]**3, -P*K_prob[i][j]**3])
                    mpms_prob[i][j] = float(r[np.isreal(r)][0])
                    mass_prob[i][j] = mpms_prob[i][j]*Mstar_prob[i][j]*MSUN/MEARTH
                    radius_prob[i][j] = mr.Mpost2R([mass_prob[i][j]])

                bad = check(mass_prob[i][j], radius_prob[i][j])

In [ ]:
# save these for later
def p_to_a(p, ms):
    return (p**2*BIGG*ms/(4*pi**2))**(1/3)

def petrovich(i, j, mp, ms, d=d):
    if np.all(ms[i]==0):
        return True
    p = d['p'][d['starname']==d['starname'][i]]
    e = d['e'][d['starname']==d['starname'][i]]
    mu = mp[i]*MEARTH/(ms[i]*MSUN)
    ind = np.where(d['p']==d['p'][i])[0]
    if ind < len(p)-1:
        pin = np.min([p[ind], p[ind+1]])
        pout = np.max([p[ind], p[ind+1]])
        ein = np.min([e[ind], e[ind+1]])
        eout = np.max([e[ind], e[ind+1]])
        muin = np.min([mu[ind], mu[ind+1]])
        muout = np.max([mu[ind], mu[ind+1]])
    else:
        pin = np.min([p[len(p)-1], p[len(p)-2]])
        pout = np.max([p[len(p)-1], p[len(p)-2]])
        ein = np.min([e[len(p)-1], e[len(p)-2]])
        eout = np.max([e[len(p)-1], e[len(p)-2]])
        muin = np.min([mu[len(p)-1], mu[len(p)-2]])
        muout = np.max([mu[len(p)-1], mu[len(p)-2]])
    ain = p_to_a(pin, ms[i][j])
    aout = p_to_a(pout, ms[i][j])
    return aout*(1+ein)/(ain*(1-eout)) > \
        2.4*(max([muin, muout]))**(1/3)*np.sqrt(aout/ain) + 1.15   

In [ ]:
plt.figure(figsize=(13,8))
plt.plot(np.log10(mass_prob), np.log10(radius_prob), "k.", label='Transit')
plt.plot(np.log10(mass_prob)[RV], np.log10(radius_prob)[RV], ".", c="orange", label='RV')
plt.plot(np.log10(mass_prob)[TTV], np.log10(radius_prob)[TTV], ".", c="cornflowerblue", label='TTV')
plt.xlabel("$\log_{10}(m_p)$", fontsize=20)
plt.ylabel("$\log_{10}(r_p)$", fontsize=20)
plt.show()

# Build list of systems


In [ ]:
all_systems = []
transit_systems = []
rv_systems = []
mixed_systems = []

for i, star in enumerate(np.unique(d["starname"])):
    use = d["starname"] == star
    
    s = System()
    
    # stellar/system parameters
    s.starname = d["starname"][use][0]
    s.Mstar    = d["ms"][use][0]
    s.Mstar_err = np.sqrt(d["ms_err1"][use][0]**2 + \
                          d["ms_err2"][use][0]**2)/np.sqrt(2)
    s.Rstar    = d["rs"][use][0]
    s.Rstar_err = np.sqrt(d["rs_err1"][use][0]**2 + \
                          d["rs_err2"][use][0]**2)/np.sqrt(2)
    s.det_meth = d["sys_disc_method"][use][0]
        
    
    # measured planet properties
    s.rprs = d["rprs"][use]
    s.radii = d["rp"][use]
    s.radii_err = np.sqrt(d["rp_err1"][use]**2 + \
                        d["rp_err2"][use]**2)/np.sqrt(2)
    s.masses = d['mp'][use]
    s.masses_err = np.sqrt(d["mp_err1"][use]**2 + \
                        d["mp_err2"][use]**2)/np.sqrt(2)
    s.periods = d["p"][use]
    s.N = len(s.rprs)
    
    # arrays of probabilistic mass and radius
    s.prob_mass = mass_prob[use].swapaxes(0,1)
    s.prob_radius = radius_prob[use].swapaxes(0,1)
    
    # save to lists
    if s.N >= 3:
        all_systems.append(s)
        if s.det_meth == "Transit":
            transit_systems.append(s)
        elif s.det_meth == "Radial Velocity":
            rv_systems.append(s)
        elif s.det_meth == "Mixed":
            mixed_systems.append(s)
        else:
            raise ValueError("Detection method must be 'Transit',  'Radial Velocity', or 'Mixed'")

# Assign weights to stellar masses


In [ ]:
Mstar = []
starname = []
det_meth = []

for i, s in enumerate(all_systems):
    Mstar.append(s.Mstar)
    starname.append(s.starname)
    det_meth.append(s.det_meth)
    
Mstar = np.asarray(Mstar)
starname = np.asarray(starname)
det_meth = np.asarray(det_meth)

ms_rv = Mstar[det_meth == "Radial Velocity"]

def get_weights(arr, arr_rv, plot_on = True):
    
    # Fraction of stellar masses (any detection method) 
    # beyond RV limit
    # Visual inspection reveals only a few transit stars with 
    # masses slightly beyond RV limits
    
    low = np.sum(arr < arr_rv.min())/len(arr)
    high = np.sum(arr < arr_rv.max())/len(arr)
    
    train_x = np.sort(arr_rv + np.random.normal(loc=0, \
                scale=100000*sys.float_info.epsilon, size=len(arr_rv)))
    train_x = np.hstack([arr.min(), train_x, arr.max()])
    
    train_y = np.linspace(low, high, len(arr_rv))
    train_y = np.hstack([0, train_y, 1])
    
    spline = scipy.interpolate.PchipInterpolator(train_x, train_y)
    
    fit_x = np.linspace(train_x.min(), train_x.max(), 100)
    fit_y = spline(fit_x)
    
    win = sig.hann(49)
    win /= np.sum(win)
    
    fit_y_padded = np.pad(fit_y, (50,50), "edge")
    fit_y_smooth = sig.convolve(fit_y_padded, win, "same")[50:-50]
    
    fit_y_smooth = (fit_y_smooth-min(fit_y_smooth))/(max(fit_y_smooth)\
                                                     -min(fit_y_smooth))
    
    spline = scipy.interpolate.PchipInterpolator(fit_x, fit_y_smooth)

    if plot_on:
        plt.figure(figsize=(8,5))
        plt.plot(train_x, train_y, "ko")
        plt.plot(fit_x, spline(fit_x), c="orange", lw=3)
        plt.show()
    
    return arr[det_meth == "Transit"], spline(arr[det_meth == "Transit"])

ms_tr, w_tr = get_weights(Mstar, ms_rv)

# Now we can draw samples and compare cdfs


In [ ]:
# dictionaries to hold p-values for Kolmogorov-Smirnov and Anderson-Darling tests
KS = {}
AD = {}

In [ ]:
def perturbed_ks_2samp(x1, x2):
    n, m = len(x1), len(x2)
    
    D, p = stats.ks_2samp(x1, x2)

    eps_low  = np.min([D, 0.5/np.sqrt(n*m)])
    eps_high = np.min([1-D, 0.5/np.sqrt(n*m)])
    
    D_new = D + np.random.uniform(-eps_low, eps_high)
    p_new = 2*np.exp(-(2*m*D_new**2)/(1+m/n))
    
    return D_new, p_new


def extrapolated_anderson_2samp(x1, x2):
    A, cv, p = stats.anderson_ksamp([x1, x2])

    alpha = np.array([0.25, 0.10, 0.05, 0.025, 0.01, 0.005, 0.001])
    
    res_fxn = lambda theta, x, y: y - 1/(1 + theta[0]*np.exp(x/theta[1]))
    
    fit, success = op.leastsq(res_fxn, [0.25, 1], args=(cv, alpha))
    
    z = np.linspace(-2,6)
    p_new = 1/(1 + fit[0]*np.exp(A/fit[1]))
    
    return A, p_new

# Here's the plotting function

In [ ]:
def quick_plot(data, ks, ad, datalabel, gridsize=1000, colors=["k", "r"], labels=["Transit", "RV"]):    
    
    if colors is None:
        colors = []
        for k in range(len(data)):
            colors.append("C{0}".format(k))
    
    fig, ax = plt.subplots(1,2, figsize=(16,5))        
    
    for k, x in enumerate(data):

        X = np.linspace(x.min(), x.max(), gridsize)
        Y = np.zeros((x.shape[0], gridsize))

        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                Y[i,j] = np.sum(x[i] <= X[j])

        Y /= x.shape[1]

        Y_mean = Y.mean(0)
        Y_up = Y_mean + Y.std(0)
        Y_dn = Y_mean - Y.std(0)

        Y_up[Y_up > 1.0] = 1.0
        Y_dn[Y_dn < 0.0] = 0.0
        
        
        # enforce monotonicity of bounds
        for i in range(1,gridsize):
            if Y_up[i] < Y_up[i-1]:
                Y_up[i] = Y_up[i-1]
        
        for i in range(1,gridsize):
            if Y_dn[gridsize-i-1] > Y_dn[gridsize-i]:
                Y_dn[gridsize-i-1] = Y_dn[gridsize-i]

        ax[0].plot(X, Y_mean, c=colors[k], label=labels[k])
        ax[0].plot(X, Y_up, c=colors[k], ls=":")
        ax[0].plot(X, Y_dn, c=colors[k], ls=":")
        ax[0].fill_between(X, Y_up, Y_dn, color=colors[k], alpha=0.1)
        ax[0].legend(fontsize=16)
        ax[0].set_xlabel(datalabel, fontsize=20)
        ax[0].set_ylabel("cdf", fontsize=20)
        
    ax[1].hist(np.log10(ks), color="cornflowerblue", histtype="step", lw=3, label="KS")
    ax[1].hist(np.log10(ad), color="orange", histtype="step", lw=3, label="AD")
    ax[1].axvline(np.log10(np.mean(ks)), color="navy", lw=3, ls="--")
    ax[1].axvline(np.log10(np.mean(ad)), color="red", lw=3, ls="--")
    ax[1].set_xlabel("p-value", fontsize=20)
    ax[1].set_yticks([])
    ax[1].legend(loc="upper left", fontsize=16)


    return fig, ax

# Dynamical mass, $\mu$


In [ ]:
Nsys = len(ms_rv)

KS["mu"] = np.zeros(NBOOT)
AD["mu"] = np.zeros(NBOOT)

mu_rv = np.zeros((NBOOT,Nsys))
mu_tr = np.zeros((NBOOT,Nsys))

for i in range(NBOOT):
    for j in range(Nsys):
        # use all RV systems
        mu_rv[i,j] = archinfo.mu(rv_systems[j].prob_mass[i], rv_systems[j].Mstar)
        
        # do a weighted draw of a random transit system
        w_rand = np.random.uniform(0,1)
        loc = np.argmin(np.abs(w_rand-w_tr))
        mu_tr[i,j] = archinfo.mu(transit_systems[loc].prob_mass[i], transit_systems[loc].Mstar)
    
    mu_rv[i] = np.sort(mu_rv[i])
    mu_tr[i] = np.sort(mu_tr[i])
    
    # p-values
    KS["mu"][i] = perturbed_ks_2samp(mu_rv[i], mu_tr[i])[1]
    AD["mu"][i] = extrapolated_anderson_2samp(mu_rv[i], mu_tr[i])[1]

logmu_rv = np.log10(mu_rv)
logmu_tr = np.log10(mu_tr)


fig, ax = quick_plot([logmu_tr, logmu_rv], KS["mu"], AD["mu"], "Dynamical Mass, $\mu$")

# Normalized dynamical mass, $\mu/N$


In [ ]:
Nsys = len(ms_rv)

KS["mu_N"] = np.zeros(NBOOT)
AD["mu_N"] = np.zeros(NBOOT)

mu_N_rv = np.zeros((NBOOT,Nsys))
mu_N_tr = np.zeros((NBOOT,Nsys))

for i in range(NBOOT):
    for j in range(Nsys):
        # use all RV systems
        mu_N_rv[i,j] = archinfo.mu(rv_systems[j].prob_mass[i], rv_systems[j].Mstar)/rv_systems[j].N
        
        # do a weighted draw of a random transit system
        w_rand = np.random.uniform(0,1)
        loc = np.argmin(np.abs(w_rand-w_tr))
        mu_N_tr[i,j] = archinfo.mu(transit_systems[loc].prob_mass[i], \
                                   transit_systems[loc].Mstar)/transit_systems[loc].N

    mu_N_rv[i] = np.sort(mu_N_rv[i])
    mu_N_tr[i] = np.sort(mu_N_tr[i])
    
    # p-values 
    KS["mu_N"][i] = perturbed_ks_2samp(mu_N_rv[i], mu_N_tr[i])[1]
    AD["mu_N"][i] = extrapolated_anderson_2samp(mu_N_rv[i], mu_N_tr[i])[1]


logmu_N_rv = np.log10(mu_N_rv)
logmu_N_tr = np.log10(mu_N_tr)


fig, ax = quick_plot([logmu_N_tr, logmu_N_rv], KS["mu_N"], AD["mu_N"], \
                     "Normalized Dynamical Mass, $\mu/N$")

# Mass partitioning, Q


In [ ]:
Nsys = len(ms_rv)

KS["Q"] = np.zeros(NBOOT)
AD["Q"] = np.zeros(NBOOT)

Q_rv = np.zeros((NBOOT,Nsys))
Q_tr = np.zeros((NBOOT,Nsys))

for i in range(NBOOT):
    for j in range(Nsys):
        # use all RV systems
        Q_rv[i,j] = archinfo.Q(rv_systems[j].prob_mass[i])
        
        # do a weighted draw of a random transit system
        w_rand = np.random.uniform(0,1)
        loc = np.argmin(np.abs(w_rand-w_tr))
        
        Q_tr[i,j] = archinfo.Q(transit_systems[loc].prob_mass[i])


    Q_rv[i] = np.sort(Q_rv[i])
    Q_tr[i] = np.sort(Q_tr[i])
    
    # p-values
    KS["Q"][i] = perturbed_ks_2samp(Q_rv[i], Q_tr[i])[1]
    AD["Q"][i] = extrapolated_anderson_2samp(Q_rv[i], Q_tr[i])[1]
        
        
fig, ax = quick_plot([Q_tr, Q_rv], KS["Q"], AD["Q"], "Mass Partitioning, $Q$")

# Monotonicity, M


In [ ]:
Nsys = len(ms_rv)

KS["M"] = np.zeros(NBOOT)
AD["M"] = np.zeros(NBOOT)

M_rv = np.zeros((NBOOT,Nsys))
M_tr = np.zeros((NBOOT,Nsys))

for i in range(NBOOT):
    for j in range(Nsys):
        # use all RV systems
        M_rv[i,j] = archinfo.M(rv_systems[j].periods, rv_systems[j].prob_mass[i])
        
        # do a weighted draw of a random transit system
        w_rand = np.random.uniform(0,1)
        loc = np.argmin(np.abs(w_rand-w_tr))
        M_tr[i,j] = archinfo.M(transit_systems[loc].periods, transit_systems[loc].prob_mass[i])
    

    M_rv[i] = np.sort(M_rv[i])
    M_tr[i] = np.sort(M_tr[i])
    
    # p-values
    KS["M"][i] = perturbed_ks_2samp(M_rv[i], M_tr[i])[1]
    AD["M"][i] = extrapolated_anderson_2samp(M_rv[i], M_tr[i])[1]
        
fig, ax = quick_plot([M_tr, M_rv], KS["M"], AD["M"], "Monotonicity, $M$")

# Gap complexity, C


In [ ]:
Nsys = len(ms_rv)

KS["C"] = np.zeros(NBOOT)
AD["C"] = np.zeros(NBOOT)

C_rv = np.zeros((NBOOT,Nsys))
C_tr = np.zeros((NBOOT,Nsys))

for i in range(NBOOT):
    for j in range(Nsys):
        # use all RV systems
        C_rv[i,j] = archinfo.C(rv_systems[j].periods)
        
        # do a weighted draw of a random transit system
        w_rand = np.random.uniform(0,1)
        loc = np.argmin(np.abs(w_rand-w_tr))
        C_tr[i,j] = archinfo.C(transit_systems[loc].periods)


    C_rv[i] = np.sort(C_rv[i])
    C_tr[i] = np.sort(C_tr[i])
    
    # p-values
    KS["C"][i] = perturbed_ks_2samp(C_rv[i], C_tr[i])[1]
    AD["C"][i] = extrapolated_anderson_2samp(C_rv[i], C_tr[i])[1]
        
        
fig, ax = quick_plot([C_tr, C_rv], KS["C"], AD["C"], "Gap Complexity, $C$")

# Characteristic Spacing, S


In [ ]:
Nsys = len(ms_rv)

KS["S"] = np.zeros(NBOOT)
AD["S"] = np.zeros(NBOOT)

S_rv = np.zeros((NBOOT,Nsys))
S_tr = np.zeros((NBOOT,Nsys))

for i in range(NBOOT):
    for j in range(Nsys):
        # use all RV systems
        S_rv[i,j] = archinfo.S(rv_systems[j].periods, rv_systems[j].prob_mass[i], rv_systems[j].Mstar)
        
        # do a weighted draw of a random transit system
        w_rand = np.random.uniform(0,1)
        loc = np.argmin(np.abs(w_rand-w_tr))
        S_tr[i,j] = archinfo.S(transit_systems[loc].periods, transit_systems[loc].prob_mass[i], 
                               transit_systems[loc].Mstar)


    S_rv[i] = np.sort(S_rv[i])
    S_tr[i] = np.sort(S_tr[i])
    
    # p-values
    KS["S"][i] = perturbed_ks_2samp(S_rv[i], S_tr[i])[1]
    AD["S"][i] = extrapolated_anderson_2samp(S_rv[i], S_tr[i])[1]
        
        
fig, ax = quick_plot([S_tr, S_rv], KS["S"], AD["S"], "Characteristic Spacing, $S$")

# Write out parameters for Andrew

In [ ]:
MAINPATH = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/'
filepath = MAINPATH + 'Mstar.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'Mstar_mean']
    for i in range(NBOOT):
        header = np.concatenate((header, ['Mstar_%d' %i]))
    writer.writerow(header)
    
    for i in range(len(Mstar_prob)):
        arr = np.concatenate(([d['planet_id'][i]], [d['ms'][i]], \
                              Mstar_prob[i]))
        writer.writerow(arr)

In [ ]:
filepath = MAINPATH + 'Rstar.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'Rstar_mean']
    for i in range(NBOOT):
        header = np.concatenate((header, ['Rstar_%d' %i]))
    writer.writerow(header)
    
    for i in range(len(Mstar_prob)):
        arr = np.concatenate(([d['planet_id'][i]], [d['rs'][i]], \
                              Rstar_prob[i]))
        writer.writerow(arr)

In [ ]:
filepath = MAINPATH + 'rprs.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'rprs_mean']
    for i in range(NBOOT):
        header = np.concatenate((header, ['rprs_%d' %i]))
    writer.writerow(header)
    
    for i in range(len(Mstar_prob)):
        arr = np.concatenate(([d['planet_id'][i]], [d['rprs'][i]], \
                              rprs_prob[i]))
        writer.writerow(arr)

In [ ]:
filepath = MAINPATH + 'semi_amplitudes.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'K_mean']
    for i in range(NBOOT):
        header = np.concatenate((header, ['K_%d' %i]))
    writer.writerow(header)
    
    for i in range(len(Mstar_prob)):
        arr = np.concatenate(([d['planet_id'][i]], [d['K'][i]], \
                              K_prob[i]))
        writer.writerow(arr)

In [ ]:
filepath = MAINPATH + 'e.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'e']
    writer.writerow(header)
    
    for i in range(len(d['e'])):
        arr = np.concatenate(([d['planet_id'][i]], [d['e'][i]]))
        writer.writerow(arr)

In [ ]:
filepath = MAINPATH + 'p.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'P']
    writer.writerow(header)
    
    for i in range(len(d['p'])):
        arr = np.concatenate(([d['planet_id'][i]], [d['p'][i]]))
        writer.writerow(arr)

In [ ]:
filepath = MAINPATH + 'teff.csv'
with open(filepath, 'w') as outfile:
    writer = csv.writer(outfile)
    
    header = ['planet_id', 'Teff']
    writer.writerow(header)
    
    for i in range(len(d['teff'])):
        arr = np.concatenate(([d['planet_id'][i]], [d['teff'][i]]))
        writer.writerow(arr)

In [ ]:
print('Writing complete!')